# FIRST ORDER MODEL FOR PROPOSED ENFIELD BUILDING

In [9]:
import pandas as pd
import numpy as np
import random

In [181]:
class Person:
    """
    A class that represents a person at the food pantry.
    """
    def __init__(self, model):
        self.order = model.pressure_points.copy()
        i=0
        while i < len(self.order):
            if np.random.uniform() > self.order[i].p:
                self.order.pop(i)
            else:
                i += 1
        random.shuffle(self.order)
        self.shopping_time = 0
        for i in self.order:
            self.shopping_time += i.time
        self.enter = model.iterations
        self.available = True
        
    def setLeave(self, iterations):
        self.leave = iterations
        
    def __str__(self):
        str = "Stations remaining: "
        for i in self.order:
            str += ", " + i.name
        return str
        
class Pressure_Point:
    """
    A class that represents a pressure point in the food pantry.
    """
    def __init__(self, name, capacity, time, p = 1):
        self.name = name
        self.capacity = capacity
        self.time = time
        self.atCapacity = False
        self.people = []
        self.timeRemaining = []
        self.p = p
        
    def update(self):
        i = 0
        while i < len(self.timeRemaining):
            self.timeRemaining[i] -= 1
            if self.timeRemaining[i] == 0:
                self.timeRemaining.pop(i)
                self.people[i].available = True
                self.people.pop(i)
                self.atCapacity = False
            else:
                i += 1
            
    def add(self, person):
        if not self.atCapacity:
            person.available = False
            self.people.append(person)
            self.timeRemaining.append(self.time)
            person.order.pop(0)
            if len(self.people) == self.capacity:
                self.atCapacity = True
    
    def __str__(self):
        #return "Number of People: " + str(len(self.people))
        return "Time Remaining: " + str(self.timeRemaining)

class Model:
    """
    A class that represents a simulation model.
    """
    # ATTRIBUTES:
    # Attribute cars: the number of cars that arrive at the food pantry daily
    # Invariant: cars is an int or float >= 0
    #
    # Attribute per_car: the number of adults in each car
    # Invariant: per_car is an int > 0
    #
    # Attribute hours_open: the number of hours open per day
    # Invariant: hours_open is an int or float >= 0
    #
    # Attribute arrival_rate: the number of people who arrive at the food pantry per minute
    # Invariant: arrival_rate is an int or float >= 0
    #
    # Attribute pressure_points: the pressure_points in the food pantry
    # Invariant: pressure_points is a list of Pressure_Point objects
    #
    # Attribute iterations: how many minutes have been simulated in the model
    # Invariant: iterations is an int >= 0
    #
    # Attribute people: the people currently in the food pantry
    # Invariant: people is a list of People objects
    
    def setPressurePoints(self, names, times, capacities, ps):
        assert len(names) == len(times)
        assert len(names) == len(capacities)
        
        self.pressure_points = []
        
        for i in range(len(names)):
            self.pressure_points.append(Pressure_Point(names[i], capacities[i], times[i], ps[i]))
    
    def __init__(self, cars, per_car, hours_open, names, times, capacities, ps):
        """
        Initializes a model.
        """
        self.cars = cars
        self.per_car = per_car
        self.arrival_rate = cars * per_car / (hours_open * 60)
        self.hours_open = hours_open
        self.setPressurePoints(names, times, capacities, ps)
        self.iterations = 0
        self.arrival_count = 0
        self.people = []
        self.stats = []
        self.arrival_stats = []
        self.people_arrived = 0
        self.day = 0
        self.day_df = pd.DataFrame(columns = ['enter', 'leave'])
        
    def iterate(self):
        """
        Performs the actions of one minute in the food pantry.
        """
        self.iterations += 1
        self.arrival_count += self.arrival_rate
        while self.arrival_count >= 1:
            self.people.append(Person(self))
            self.arrival_count -= 1
            self.people_arrived += 1
            
        for i in self.pressure_points:
            i.update()
        
        i = 0
        while i < len(self.people):
            if len(self.people[i].order) == 0 and self.people[i].available:
                self.people[i].setLeave(self.iterations)
                self.day_df.loc[len(self.day_df.index)] = [self.day, self.people[i].enter, self.people[i].leave, 
                                                           self.people[i].leave - self.people[i].enter,
                                                           self.people[i].shopping_time, 
                                                           self.people[i].leave - self.people[i].enter - self.people[i].shopping_time]
                self.people.pop(i)
            else:
                if self.people[i].available:
                    self.people[i].order[0].add(self.people[i])
                i += 1
                
        #self.print_state()
                
    def simulate_day(self):
        """
        Runs simulation for one day.
        """
        self.day += 1
        i = 0
        self.arrival_count = 0
        self.people_arrived = 0
        self.people = []
        self.day_df = pd.DataFrame(columns = ['day', 'enter', 'leave', 'total_time', 'shopping_time', 'waiting_time'])
        while i < self.hours_open * 60:
            i += 1
            self.iterate()
        self.stats.append(self.day_df)
        self.arrival_stats.append(self.people_arrived)
        
    def print_state(self):
            print("PEOPLE:")
            for i in self.people:
                print(str(i))
            print("PRESSURE POINTS:")
            for i in self.pressure_points:
                print(str(i))
                
    def simulate(self, days):
        for i in range(days):
            self.simulate_day()

In [182]:
cars = 65
per_car = 1
hours_open = 3
names = ["rescued produce", "assorted meats", "baked goods", "clothes"]
times = [5, 5, 5, 10]
capacities = [2, 2, 2, 7]
ps = [1, 1, 1, .25]
enfield = Model(cars, per_car, hours_open, names, times, capacities, ps)

In [183]:
enfield.simulate(14)

In [184]:
df = pd.DataFrame(columns = ['day',
                             'shoppers_served',
                             'shoppers_arrived',
                             'avg_total_time',
                             'avg_shopping_time',
                             'avg_waiting_time'])
for i in range(len(enfield.stats)):
    df.loc[len(df.index)] = [int(enfield.stats[i]['day'][0]),
                             len(enfield.stats[i].index),
                             enfield.arrival_stats[i],
                             enfield.stats[i]['total_time'].mean(),
                             enfield.stats[i]['shopping_time'].mean(),
                             enfield.stats[i]['waiting_time'].mean()]
df

,day,shoppers_served,shoppers_arrived,avg_total_time,avg_shopping_time,avg_waiting_time
0,1.0,54.0,65.0,23.629630,16.851852,6.777778
1,2.0,55.0,65.0,22.000000,16.636364,5.363636
2,3.0,55.0,65.0,24.418182,18.090909,6.327273
3,4.0,54.0,65.0,24.611111,16.666667,7.944444
4,5.0,56.0,65.0,23.125000,17.321429,5.803571
5,6.0,53.0,65.0,26.056604,17.830189,8.226415
6,7.0,56.0,65.0,22.785714,17.321429,5.464286
7,8.0,54.0,65.0,25.833333,17.592593,8.240741
8,9.0,53.0,65.0,27.000000,17.452830,9.547170
9,10.0,54.0,65.0,26.666667,18.518519,8.148148
